In [25]:
import pandas as pd
import numpy as np

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import mean_absolute_error, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, accuracy_score

In [3]:
# Load dataset
movies_df = pd.read_csv('../data/clean/clean_movies.csv')

In [5]:
movies_df = pd.read_csv('../data/clean/clean_movies.csv')
numeric_data = ['vote_average', 'revenue', 'runtime', 'budget']
print(movies_df.columns)

scaler = MinMaxScaler()
movies_df[numeric_data] = scaler.fit_transform(movies_df[numeric_data])
movies_df['adult_true'] = movies_df['adult'].astype(int)
movies_df['adult_false'] = (~movies_df['adult']).astype(int)
movies_df=movies_df.drop(columns=['adult'])

# Crear columnas para mes y día
movies_df['release_month'] = pd.to_datetime(movies_df['release_date']).dt.month
movies_df['release_day'] = pd.to_datetime(movies_df['release_date']).dt.day
# Eliminar la columna 'release_date' original si ya no la necesitas
movies_df.drop(columns=['release_date'], inplace=True)

Index(['Unnamed: 0', 'title', 'vote_average', 'release_date', 'revenue',
       'runtime', 'adult', 'budget', 'genres'],
      dtype='object')


In [11]:
movies_df_clean =movies_df.drop(columns=['Unnamed: 0'])

In [13]:
movies_df_clean

,title,vote_average,revenue,runtime,budget,genres,adult_true,adult_false,release_month,release_day
0,Avatar,0.7573,1.000000,0.136223,0.515217,"Action, Adventure, Fantasy, Science Fiction",0,1,12,15
1,Avengers: Endgame,0.8263,0.957689,0.155831,0.773913,"Adventure, Science Fiction, Action",0,1,4,24
2,Avatar: The Way of Water,0.7654,0.793599,0.167183,1.000000,"Science Fiction, Adventure, Action",0,1,12,14
3,Star Wars: The Force Awakens,0.7293,0.707398,0.109391,0.532609,"Adventure, Action, Science Fiction",0,1,12,15
4,Avengers: Infinity War,0.8255,0.701991,0.122807,0.652174,"Adventure, Action, Science Fiction",0,1,4,25
...,...,...,...,...,...,...,...,...,...,...
5892,Shrek's Yule Log,1.0000,0.000000,0.000000,0.000002,"Family, Animation, Comedy",0,1,12,7
5893,Grandpa Comes Back from Germany,0.0000,0.000000,0.077399,0.000001,"Crime, Thriller, Action",0,1,8,9
5894,Across the Rails,0.0000,0.000000,0.010320,0.000007,"Documentary, Adventure",0,1,1,12
5895,Plane Tree,0.5000,0.000000,0.071207,0.000435,"Family, Drama, Mystery",0,1,10,5


In [19]:
# Features: Select numeric columns and genre dummy variables
X = movies_df[["vote_average", "revenue", "runtime", "budget", "adult_true", "adult_false"]]

# Target: Release month
y = movies_df["release_month"]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [21]:
# Train a Random Forest classifier
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [27]:
# Predictions
y_pred = rf_model.predict(X_test)

# Evaluation
print("Classification Report:")
print(classification_report(y_test, y_pred))
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")


Classification Report:
              precision    recall  f1-score   support

           1       0.11      0.11      0.11        98
           2       0.10      0.12      0.11        94
           3       0.07      0.05      0.06        91
           4       0.08      0.09      0.09        76
           5       0.15      0.14      0.14        85
           6       0.14      0.11      0.12       109
           7       0.07      0.06      0.07        81
           8       0.07      0.07      0.07        82
           9       0.13      0.16      0.14       147
          10       0.15      0.16      0.16       124
          11       0.09      0.08      0.08        92
          12       0.15      0.17      0.16       101

    accuracy                           0.12      1180
   macro avg       0.11      0.11      0.11      1180
weighted avg       0.11      0.12      0.11      1180

Accuracy: 0.1153


In [29]:
# Feature importance
feature_importances = pd.Series(rf_model.feature_importances_, index=X.columns)
print("Feature Importances:")
print(feature_importances.sort_values(ascending=False))

# Predicting the most likely release month
predicted_month = rf_model.predict([[0.8, 0.9, 120, 0.5, 0, 1]])  # Example input
print(f"Predicted Release Month: {predicted_month[0]}")

Feature Importances:
revenue         0.274120
vote_average    0.258520
budget          0.238711
runtime         0.226912
adult_false     0.000876
adult_true      0.000860
dtype: float64
Predicted Release Month: 12


C:\Users\vasco\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
